In [1]:
! unzip sms+spam+collection.zip

Archive:  sms+spam+collection.zip
  inflating: SMSSpamCollection       
  inflating: readme                  


In [2]:
import pandas as pd
import numpy as np
import re
import nltk
nltk.download("all")

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/abc.zip.
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/alpino.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping
[nltk_data]    |       taggers/averaged_perceptron_tagger_ru.zip.
[nltk_data]    | Downloading package basque_grammars to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping grammars/basque_grammars.zip.
[nltk_data]    | Downloading package bcp47 to /root/nltk_data...
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   U

True

In [3]:
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

In [4]:
Lemma = WordNetLemmatizer()
Stem = PorterStemmer()
corpus = []

In [5]:
messages = pd.read_csv('SMSSpamCollection' , sep = "\t" , names = ["label" , "message"])

In [6]:
messages

,label,message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...
5568,ham,Will ü b going to esplanade fr home?
5569,ham,"Pity, * was in mood for that. So...any other s..."
5570,ham,The guy did some bitching but I acted like i'd...


In [7]:
for i in range(len(messages)):
  review = re.sub("[^a-zA-Z]" , " " , messages['message'][i])
  review = review.lower()
  review = review.split()
  review = [Lemma.lemmatize(word) for word in review if word not in stopwords.words("english")]
  corpus.append(" ".join(review))

In [8]:
len(corpus)

5572

In [9]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features = 5000)
X = cv.fit_transform(corpus).toarray()
X , len(X) , len(X[0])
# Metrix , No.of Sentences , No.of words

(array([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]]),
 5572,
 5000)

In [10]:
messages['label'].head()

0     ham
1     ham
2    spam
3     ham
4     ham
Name: label, dtype: object

In [11]:
y = pd.get_dummies(messages['label'])
y

,ham,spam
0,True,False
1,True,False
2,False,True
3,True,False
4,True,False
...,...,...
5567,False,True
5568,True,False
5569,True,False
5570,True,False


In [12]:
y = y.iloc[:,1].values
y

array([False, False,  True, ..., False, False, False])

In [13]:
from sklearn.model_selection import train_test_split
X_train , X_test , y_train , y_test = train_test_split(X , y , test_size = 0.2 , random_state = 0)

In [14]:
from sklearn.naive_bayes import MultinomialNB
model = MultinomialNB()
model.fit(X , y)


MultinomialNB()

In [15]:
y_pred = model.predict(X_test)

In [16]:
from sklearn.metrics import confusion_matrix
conf = confusion_matrix(y_test , y_pred)

In [17]:
conf

array([[951,   4],
       [  3, 157]])

In [18]:
from sklearn.metrics import accuracy_score
acc = accuracy_score(y_test , y_pred)
acc

0.9937219730941704

# Predicting own Mail

In [19]:
custom_text = "Hello , My name is Gowtham Raj. Here is 100000 cash prize for free that you can own. Click this 'link www.spam.com'"
len(custom_text)

115

In [20]:
custom_text = re.sub("[^a-zA-Z]" , " " , custom_text)
custom_text , len(custom_text)

('Hello   My name is Gowtham Raj  Here is        cash prize for free that you can own  Click this  link www spam com ',
 115)

In [21]:
custom_text = custom_text.lower()
custom_text

'hello   my name is gowtham raj  here is        cash prize for free that you can own  click this  link www spam com '

In [22]:
custom_text = custom_text.split()
custom_text , len(custom_text)

(['hello',
  'my',
  'name',
  'is',
  'gowtham',
  'raj',
  'here',
  'is',
  'cash',
  'prize',
  'for',
  'free',
  'that',
  'you',
  'can',
  'own',
  'click',
  'this',
  'link',
  'www',
  'spam',
  'com'],
 22)

In [23]:
custom_text = [Lemma.lemmatize(word) for word in custom_text if word not in stopwords.words("english")]
custom_text , len(custom_text)

(['hello',
  'name',
  'gowtham',
  'raj',
  'cash',
  'prize',
  'free',
  'click',
  'link',
  'www',
  'spam',
  'com'],
 12)

In [31]:
custom_text = [" ".join(custom_text)]
custom_text

['hello name gowtham raj cash prize free click link www spam com']

In [32]:
cv

CountVectorizer()

In [33]:
CX = corpus + custom_text


In [35]:
CX[-1]

'hello name gowtham raj cash prize free click link www spam com'

In [36]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features = 5000)
CVX = cv.fit_transform(CX).toarray()
CVX , len(X) , len(X[0])

(array([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]]),
 5572,
 5000)

In [45]:
val = CVX[-1]
val = val.reshape(1,-1)
val

In [52]:
result = model.predict(val)[0]

In [53]:
result

True

In [54]:
if result:
  print("The Custom Input given is a Spam")
else:
  print("The Custom Input given is a Ham")

The Custom Input given is a Spam
